In [1]:
import numpy as np
import geopandas as gpd
import pickle

from skgstat import Variogram, OrdinaryKriging
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

In [2]:
udf=gpd.read_file(r"hillside_inventory_LA_centrality_full_new_evacmidnorth_lodes.geojson")

In [3]:
udf_before = udf.copy()

In [4]:
print(udf.columns)

Index(['OBJECTID', 'SECT_ID', 'ST_DIR', 'ST_NAME', 'ST_FROM', 'ST_TO',
       'ST_TYPE', 'ST_SURFACE', 'SSORT2', 'SSORT1', 'LANE_MILES', 'CL_MILES',
       'CD', 'ST_LENGTH', 'ST_WIDTH', 'Shape__Length', 'source', 'pci',
       'cond_status', 'Street_Designation', 'Curbs', 'Sidewalks',
       'required_width', 'width_ratio', 'sidewalk_ratio', 'curb_ratio',
       'rating', 'centrality_distance', 'centrality_population',
       'centrality_time', 'centrality_width', 'centroid_lat', 'centroid_lon',
       'DAC', 'BufferYN', 'lowincome', 'scagfldzn', 'scagsubty',
       'scaganualhazardflood', 'Burn_Prob', 'mean_left_side', 'std_left_side',
       'min_left_side', 'max_left_side', 'mean_right_side', 'std_right_side',
       'min_right_side', 'max_right_side', 'street_slope_per_length',
       'street_slope_absolute', 'currently_withdrawn', 'prev_withdrawn',
       'prev_withdrawn_unmaintained', 'unmaintained_add',
       'prev_withdrawn_maintained', 'maintained', 'landslide_zone',
       

In [5]:
# print unique values in udf['Street_Designation'] & its length

print(udf['Street_Designation'].unique())
print(len(udf['Street_Designation'].unique()))

['Local Street - Standard' 'Collector' 'Avenue I' 'Boulevard II'
 'Avenue II' 'Unidentified' 'Avenue III' 'Modified Avenue II'
 'Modified Avenue I' 'Modified Boulevard II' None 'Private' 'Boulevard I'
 'Mountain Collector' 'Modified Collector' 'Hillside Collector'
 'Scenic Parkway' 'Modified Avenue III'
 'Modified Scenic Arterial Mountain' 'Modified Local Street - Standard']
20


In [6]:
for designation in udf['Street_Designation'].unique():
    print(designation)

Local Street - Standard
Collector
Avenue I
Boulevard II
Avenue II
Unidentified
Avenue III
Modified Avenue II
Modified Avenue I
Modified Boulevard II
None
Private
Boulevard I
Mountain Collector
Modified Collector
Hillside Collector
Scenic Parkway
Modified Avenue III
Modified Scenic Arterial Mountain
Modified Local Street - Standard


In [7]:
# S000_adjusted

for designation in udf['Street_Designation'].unique():
    print(designation)
    x1 = udf[(udf['Street_Designation'] == designation) & (udf['S000_adjusted'] != 0)]['centroid_lat'].to_numpy()
    y1 = udf[(udf['Street_Designation'] == designation) & (udf['S000_adjusted'] != 0)]['centroid_lon'].to_numpy()
    z_S000 = udf[(udf['Street_Designation'] == designation) & (udf['S000_adjusted'] != 0)]['S000_adjusted'].to_numpy()

    x2 = udf[(udf['Street_Designation'] == designation) & (udf['S000_adjusted'] == 0)]['centroid_lat'].to_numpy()
    y2 = udf[(udf['Street_Designation'] == designation) & (udf['S000_adjusted'] == 0)]['centroid_lon'].to_numpy()

    if len(x1) == 0:
        print("No data for designation: ", designation)

    else:
        if len(np.unique(z_S000)) == 1:
            print("All values in z1 are constant for designation: ", designation)
        else:
            print("Values in z1 are not constant.")
            variogram_model_S000 = Variogram(np.vstack((x1, y1)).T, z_S000)

            ok_S000 = OrdinaryKriging(variogram_model_S000, min_points=2, max_points=10, mode='exact')

            field_S000 = ok_S000.transform(x2, y2)

            print(str(len(field_S000)) + " values in field for designation: ", designation)
            print(str(len(x2)) + " should ideally be updated for designation: ", designation)

            # norm = Normalize(vmin=min(np.min(z1), np.min(field)), vmax=max(np.max(z1), np.max(field)))

            if len(x2) == len(field_S000):
                print("Lengths match for designation: ", designation)
                for i in range(len(x2)):
                    udf.loc[(udf['centroid_lat'] == x2[i]) & (udf['centroid_lon'] == y2[i]), 'S000_adjusted'] = field_S000[i]

            else:
                print("Lengths do not match for designation: ", designation)
                # for i in range(len(x2)):
                #     udf.loc[(udf['centroid_lat'] == x2[i]) & (udf['centroid_lon'] == y2[i]), 'S000_adjusted'] = field_S000[i]


Local Street - Standard
Values in z1 are not constant.


910 values in field for designation:  Local Street - Standard
910 should ideally be updated for designation:  Local Street - Standard
Lengths match for designation:  Local Street - Standard
Collector
Values in z1 are not constant.
43 values in field for designation:  Collector
43 should ideally be updated for designation:  Collector
Lengths match for designation:  Collector
Avenue I
Values in z1 are not constant.
19 values in field for designation:  Avenue I
19 should ideally be updated for designation:  Avenue I
Lengths match for designation:  Avenue I
Boulevard II
Values in z1 are not constant.
27 values in field for designation:  Boulevard II
27 should ideally be updated for designation:  Boulevard II
Lengths match for designation:  Boulevard II
Avenue II
Values in z1 are not constant.
25 values in field for designation:  Avenue II
25 should ideally be updated for designation:  Avenue II
Lengths match for designation:  Avenue II
Unidentified
Values in z1 are not constant.
10 values 

/Users/junwonchoi/.local/share/virtualenvs/Task_1_Graph_Junwon-uzoEHbxJ/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/junwonchoi/.local/share/virtualenvs/Task_1_Graph_Junwon-uzoEHbxJ/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/junwonchoi/.local/share/virtualenvs/Task_1_Graph_Junwon-uzoEHbxJ/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [8]:
print("Before Kriging - number of rows with 0 in S000_adjusted: ", len(udf_before[(udf_before['S000_adjusted'] == 0)]))
print("After Kriging - number of rows with 0 in S000_adjusted: ", len(udf[(udf['S000_adjusted'] == 0)]))

Before Kriging - number of rows with 0 in S000_adjusted:  1081
After Kriging - number of rows with 0 in S000_adjusted:  14


In [11]:
# All other adjusted columns

adj_cols = ['SA01_adjusted', 'SA02_adjusted', 'SA03_adjusted', 'SE01_adjusted', 'SE02_adjusted', 'SE03_adjusted', 'SI01_adjusted', 'SI02_adjusted', 'SI03_adjusted']

for col in adj_cols:
    print("Currently updating: ", col)
    for designation in udf['Street_Designation'].unique():
        print(designation)
        x1 = udf[(udf['Street_Designation'] == designation) & (udf[col] != 0)]['centroid_lat'].to_numpy()
        y1 = udf[(udf['Street_Designation'] == designation) & (udf[col] != 0)]['centroid_lon'].to_numpy()
        z1 = udf[(udf['Street_Designation'] == designation) & (udf[col] != 0)][col].to_numpy()

        x2 = udf[(udf['Street_Designation'] == designation) & (udf[col] == 0)]['centroid_lat'].to_numpy()
        y2 = udf[(udf['Street_Designation'] == designation) & (udf[col] == 0)]['centroid_lon'].to_numpy()

        if len(x1) == 0:
            print("No data for designation: ", designation)

        else:
            if len(np.unique(z1)) == 1:
                print("All values in z1 are constant for designation: ", designation)
            else:
                print("Values in z1 are not constant.")
                variogram_model = Variogram(np.vstack((x1, y1)).T, z1)

                ok = OrdinaryKriging(variogram_model, min_points=2, max_points=10, mode='exact')

                field = ok.transform(x2, y2)

                print(str(len(field)) + " values in field for designation: ", designation)
                print(str(len(x2)) + " should ideally be updated for designation: ", designation)

                # norm = Normalize(vmin=min(np.min(z1), np.min(field)), vmax=max(np.max(z1), np.max(field)))

                if len(x2) == len(field):
                    print("Lengths match for designation: ", designation)
                    for i in range(len(x2)):
                        udf.loc[(udf['centroid_lat'] == x2[i]) & (udf['centroid_lon'] == y2[i]), col] = field[i]

                else:
                    print("Lengths do not match for designation: ", designation)

Currently updating:  SA01_adjusted
Local Street - Standard
Values in z1 are not constant.
1618 values in field for designation:  Local Street - Standard
1618 should ideally be updated for designation:  Local Street - Standard
Lengths match for designation:  Local Street - Standard
Collector
Values in z1 are not constant.
108 values in field for designation:  Collector
108 should ideally be updated for designation:  Collector
Lengths match for designation:  Collector
Avenue I
Values in z1 are not constant.
23 values in field for designation:  Avenue I
23 should ideally be updated for designation:  Avenue I
Lengths match for designation:  Avenue I
Boulevard II
Values in z1 are not constant.
29 values in field for designation:  Boulevard II
29 should ideally be updated for designation:  Boulevard II
Lengths match for designation:  Boulevard II
Avenue II
Values in z1 are not constant.
32 values in field for designation:  Avenue II
32 should ideally be updated for designation:  Avenue II
Le

/Users/junwonchoi/.local/share/virtualenvs/Task_1_Graph_Junwon-uzoEHbxJ/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/junwonchoi/.local/share/virtualenvs/Task_1_Graph_Junwon-uzoEHbxJ/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Values in z1 are not constant.
0 values in field for designation:  Hillside Collector
0 should ideally be updated for designation:  Hillside Collector
Lengths match for designation:  Hillside Collector
Scenic Parkway
Values in z1 are not constant.
2 values in field for designation:  Scenic Parkway
2 should ideally be updated for designation:  Scenic Parkway
Lengths match for designation:  Scenic Parkway
Modified Avenue III
Values in z1 are not constant.
0 values in field for designation:  Modified Avenue III
0 should ideally be updated for designation:  Modified Avenue III
Lengths match for designation:  Modified Avenue III
Modified Scenic Arterial Mountain
Values in z1 are not constant.
1 values in field for designation:  Modified Scenic Arterial Mountain
1 should ideally be updated for designation:  Modified Scenic Arterial Mountain
Lengths match for designation:  Modified Scenic Arterial Mountain
Modified Local Street - Standard
Values in z1 are not constant.
0 values in field for d

/Users/junwonchoi/.local/share/virtualenvs/Task_1_Graph_Junwon-uzoEHbxJ/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Values in z1 are not constant.
1053 values in field for designation:  Local Street - Standard
1053 should ideally be updated for designation:  Local Street - Standard
Lengths match for designation:  Local Street - Standard
Collector
Values in z1 are not constant.
60 values in field for designation:  Collector
60 should ideally be updated for designation:  Collector
Lengths match for designation:  Collector
Avenue I
Values in z1 are not constant.
26 values in field for designation:  Avenue I
26 should ideally be updated for designation:  Avenue I
Lengths match for designation:  Avenue I
Boulevard II
Values in z1 are not constant.
29 values in field for designation:  Boulevard II
29 should ideally be updated for designation:  Boulevard II
Lengths match for designation:  Boulevard II
Avenue II
Values in z1 are not constant.
29 values in field for designation:  Avenue II
29 should ideally be updated for designation:  Avenue II
Lengths match for designation:  Avenue II
Unidentified
Values i

...

/Users/junwonchoi/.local/share/virtualenvs/Task_1_Graph_Junwon-uzoEHbxJ/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/junwonchoi/.local/share/virtualenvs/Task_1_Graph_Junwon-uzoEHbxJ/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Values in z1 are not constant.
0 values in field for designation:  Modified Local Street - Standard
0 should ideally be updated for designation:  Modified Local Street - Standard
Lengths match for designation:  Modified Local Street - Standard
Currently updating:  SI02_adjusted
Local Street - Standard
Values in z1 are not constant.
1842 values in field for designation:  Local Street - Standard
1842 should ideally be updated for designation:  Local Street - Standard
Lengths match for designation:  Local Street - Standard
Collector
Values in z1 are not constant.
122 values in field for designation:  Collector
122 should ideally be updated for designation:  Collector
Lengths match for designation:  Collector
Avenue I
Values in z1 are not constant.
34 values in field for designation:  Avenue I
34 should ideally be updated for designation:  Avenue I
Lengths match for designation:  Avenue I
Boulevard II
Values in z1 are not constant.
30 values in field for designation:  Boulevard II
30 shoul

/Users/junwonchoi/.local/share/virtualenvs/Task_1_Graph_Junwon-uzoEHbxJ/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/junwonchoi/.local/share/virtualenvs/Task_1_Graph_Junwon-uzoEHbxJ/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/junwonchoi/.local/share/virtualenvs/Task_1_Graph_Junwon-uzoEHbxJ/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Values in z1 are not constant.
943 values in field for designation:  Local Street - Standard
943 should ideally be updated for designation:  Local Street - Standard
Lengths match for designation:  Local Street - Standard
Collector
Values in z1 are not constant.
45 values in field for designation:  Collector
45 should ideally be updated for designation:  Collector
Lengths match for designation:  Collector
Avenue I
Values in z1 are not constant.
19 values in field for designation:  Avenue I
19 should ideally be updated for designation:  Avenue I
Lengths match for designation:  Avenue I
Boulevard II
Values in z1 are not constant.
29 values in field for designation:  Boulevard II
29 should ideally be updated for designation:  Boulevard II
Lengths match for designation:  Boulevard II
Avenue II
Values in z1 are not constant.
26 values in field for designation:  Avenue II
26 should ideally be updated for designation:  Avenue II
Lengths match for designation:  Avenue II
Unidentified
Values in 

/Users/junwonchoi/.local/share/virtualenvs/Task_1_Graph_Junwon-uzoEHbxJ/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/junwonchoi/.local/share/virtualenvs/Task_1_Graph_Junwon-uzoEHbxJ/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/junwonchoi/.local/share/virtualenvs/Task_1_Graph_Junwon-uzoEHbxJ/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [ ]:
udf.to_file(r"hillside_inventory_LA_centrality_full_new_evacmidnorth_lodes_kriging.geojson", driver='GeoJSON')